https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [50]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [26]:
from neo4j_graphrag.generation import RagTemplate

prompt_template = RagTemplate(
    system_instructions="""당신은 전문 지식과 폭넓은 정보를 갖춘 최첨단 에이전트입니다. 사용자가 질문을 하면 다음 지침에 따라 응답하세요.
1. **풍부한 정보 제공:**
   - 질문에 대해 가능한 모든 관련 배경 지식과 맥락을 설명하고, 필요한 경우 구체적인 예시와 참고 사항을 포함하세요.
   - 복잡한 주제는 단계별로 세분화하여 설명하며, 각 단계의 중요 포인트를 명확하게 전달하세요.

2. **체계적이고 명확한 응답:**
   - 응답은 명확한 구조(예: 개요, 세부 사항, 결론)와 논리적 순서를 갖추어 작성하세요.
   - 정보의 출처나 근거가 있을 경우 이를 함께 명시하고, 최신 정보에 기반한 답변을 제공하도록 노력하세요.

3. **사용자 맞춤형 설명:**
   - 질문의 맥락과 사용자의 수준에 맞춰 설명의 깊이와 세부 사항을 조절하세요.
   - 추가 설명이나 예시가 필요한 경우, 부가 설명을 제공하여 이해를 돕도록 하세요.

4. **중립적이고 객관적인 자세:**
   - 모든 정보는 사실에 기반하여 객관적으로 전달하며, 다양한 관점이나 의견이 존재하는 경우 이를 균형 있게 소개하세요.
   - 최신 업데이트나 변화가 있는 경우, 그에 따른 주의사항을 명확히 설명하세요.

5. **대화형 인터랙션 유지:**
   - 사용자의 추가 질문이나 요청에 대해 신속하고 유연하게 대응하며, 필요한 경우 추가적인 정보를 요청하여 보다 정확한 답변을 제공할 수 있도록 하세요.

이러한 지침을 준수하여 사용자가 필요로 하는 풍부하고 심도 있는 정보를 제공하는 것이 목표입니다. 항상 사용자에게 최고의 도움이 될 수 있도록 노력하세요.

Domain url is 'https://www.lguplus.com/'.
    """
)

In [37]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "temperature": 0  # turning temperature down for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

In [38]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=prompt_template)

result = vector_rag.search("넷플릭스 관련된 요금제를 알려줘", retriever_config={'top_k': 5}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer="현재 제공된 데이터에는 넷플릭스 관련 요금제에 대한 정보가 포함되어 있지 않습니다. 그러나 다른 미디어 및 OTT 서비스 관련 요금제에 대한 정보는 다음과 같습니다:\n\n1. **V컬러링 바이브 플러스**\n   - **요금**: 월 8,800원\n   - **서비스 설명**: V컬러링과 NAVER VIBE 앱 무제한 듣기를 할인된 금액으로 이용할 수 있는 서비스\n   - **상세 주소**: [V컬러링 바이브 플러스](https://www.lguplus.com/mobile/plan/addon/addon-media/LRZ0003130)\n\n2. **아이들나라(스탠다드+러닝)**\n   - **요금**: 무료\n   - **서비스 설명**: 아이들나라 서비스의 학습, 독서 콘텐츠 전체와 놀이 콘텐츠 일부를 무료로 즐길 수 있는 '미디어 서비스' 혜택\n   - **상세 주소**: [아이들나라(스탠다드+러닝)](https://www.lguplus.com/mobile/plan/addon/addon-media/LRZ0003315)\n\n3. **티빙 베이직**\n   - **요금**: 월 9,500원\n   - **서비스 설명**: 티빙 오리지널 콘텐츠와 tvN, JTBC, Mnet 등의 최신 방송 프로그램, 스포츠, 공연 중계, 애니메이션 등을 제공하는 OTT 서비스\n   - **상세 주소**: [티빙 베이직](https://www.lguplus.com/mobile/plan/addon/addon-media/LRZ1000435)\n\n4. **티빙 프리미엄**\n   - **요금**: 월 17,000원\n   - **서비스 설명**: 티빙 오리지널 콘텐츠와 tvN, JTBC, Mnet 등의 최신 방송 프로그램, 스포츠, Apple TV+ 콘텐츠 등을 제공하는 OTT 서비스\n   - **상세 주소**: [티빙 프리미엄](https://www.lguplus.com/mobile/plan/addon/addon-media/LRZ

In [46]:
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
MATCH (node)-[relList]-{1, 2}(nb: !Document)
RETURN nb.id, relList[0]
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=prompt_template)

result = graph_rag.search("테더링 요금제에 대해 알려줘", retriever_config={'top_k': 1}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


ValueError: Expected adapter or supported vector store, but got langchain_neo4j.vectorstores.neo4j_vector.Neo4jVector

In [53]:
# Text2Cypher Retriever
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

product_schema = {
    "label": "Service",
    "properties": [
        {"name": "id", "type": "string"},
    ]
}

price_schema = {
    "label": "Price",
    "properties": [
        {"name": "id", "type": "string"},
    ]
}

has_price_schema = {
    "type": "HAS_PRICE",
    "source": "Product",
    "target": "Price"
}


neo4j_schema ={
    "nodes": [product_schema, price_schema],
    "relationships": [has_price_schema]
}

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm,
    neo4j_schema=str(neo4j_schema)
)

text2cypher_rag = GraphRAG(
    llm=llm,
    retriever=text2cypher_retriever,
    prompt_template=prompt_template
)

result = text2cypher_rag.search(
    "티빙 베이직 가격 알려줘",
    return_context=True
)
result

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='티빙 베이직 요금제의 가격은 월 9,500원입니다. 이 요금제는 티빙의 기본적인 콘텐츠를 이용할 수 있는 플랜으로, 다양한 드라마, 영화, 예능 프로그램 등을 시청할 수 있습니다. 티빙은 한국의 대표적인 OTT(Over-The-Top) 서비스 중 하나로, 다양한 장르의 콘텐츠를 제공하여 많은 사용자들에게 인기를 끌고 있습니다. 요금제에 대한 더 자세한 정보나 다른 플랜에 대한 비교가 필요하시면 추가로 문의해 주세요.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content="<Record p=<Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1012' labels=frozenset({'__Entity__', 'Price'}) properties={'id': '월 9,500원'}>>", metadata=None)], metadata={'cypher': "MATCH (s:Service {id: '티빙 베이직'})-[:HAS_PRICE]->(p:Price) RETURN p", '__retriever': 'Text2CypherRetriever'}))